# Integrating Flotorch Session Management with LangChain

This notebook demonstrates how to integrate Flotorch's session management with a LangChain agent. By using `FlotorchLangChainSession` as the memory for the `AgentExecutor`, we can create a conversational agent that remembers the context of the current session.

### Key Concepts:
- **`FlotorchLangChainLLM`**: The language model powering the agent.
- **`FlotorchLangChainSession`**: The session storage mechanism for persisting conversation history.
- **LangChain Memory**: The `AgentExecutor` is configured to use the Flotorch session backend for conversational memory.
- **`AgentExecutor` and `ChatPromptTemplate`**: The core LangChain components for building and orchestrating the AI workflow.

## 1. Setup and Configuration

The following cells install the necessary packages, configure API credentials, and import all required components from the Flotorch and LangChain libraries.

In [ ]:
# install flotorch langchain package
%pip install  flotorch[langchain]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "https://qa-gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"

In [ ]:
# Import LangChain and related modules
from flotorch.langchain.llm import FlotorchLangChainLLM
from flotorch.langchain.session import FlotorchLangChainSession
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool

print("Imported necessary libraries successfully")

## 2. Model Configuration

Here, we initialize the `FlotorchLangChainLLM` with our model configuration. This will serve as the reasoning engine for our LangChain agent.

In [ ]:
model  =  FlotorchLangChainLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )
print("FlotorchLLM initialized")

## 3. Session Storage Setup

We configure Flotorch session storage and integrate it with LangChain's memory system. This enables the agent to maintain conversational context.

In [ ]:
session_memory = FlotorchLangChainSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

print("short_term_memory initialized")

In [ ]:
@tool
def analyze_text(text: str) -> str:
    """
    Analyze text and provide comprehensive statistics.
    
    Args:
        text (str): Text to analyze
        
    Returns:
        str: Formatted text analysis with statistics
    """
    word_count = len(text.split())
    char_count = len(text)
    char_count_no_spaces = len(text.replace(' ', ''))
    sentences = text.count('.') + text.count('!') + text.count('?')
    
    return f"""Text Analysis:
- Words: {word_count}
- Characters (with spaces): {char_count}
- Characters (without spaces): {char_count_no_spaces}
- Sentences: {sentences}"""


@tool
def weather(city: str) -> str:
    """Get weather for a city. Available cities: New York, London, Tokyo, Paris."""
    city = city.lower().strip()

    weather_info = {
        "new york": "Sunny, 72°F, Humidity: 45%",
        "london": "Cloudy, 15°C, Humidity: 70%",
        "tokyo": "Rainy, 25°C, Humidity: 80%",
        "paris": "Partly cloudy, 18°C, Humidity: 60%"
    }

    if city in weather_info:
        return f"Weather in {city.title()}: {weather_info[city]}"
    else:
        return f"I don't have weather data for '{city}'. Try: New York, London, Tokyo, or Paris."

tools = [analyze_text, weather]

print("Custom tool defined successfully.")

## 4. Agent and Prompt Configuration

Next, we define the agent's logic using a `ChatPromptTemplate`. The prompt defines the agent's persona and includes a placeholder for `{history}` to inject the conversational context from our session memory.

In [ ]:
# Create the Agent
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to tools for calculations, weather, and information search.
    Use the appropriate tool when needed, or answer directly if you can.
    Keep your responses clear and concise.
    session history: {history}"""),
    ("human", "{input}"),
    ("placeholder","{agent_scratchpad}"),
])

agent = create_openai_functions_agent(model,tools,prompt)

print(f"Agent is created successfully")

## 5. Agent Executor Assembly

We create an `AgentExecutor` to run the agent. Crucially, we pass the `session_memory` object to the `memory` parameter of the executor.

In [ ]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=session_memory,
    verbose=False,
    handle_parsing_errors=True
)


## 6. Interactive Chat

This loop starts an interactive chat session. The agent will use the Flotorch-backed session memory to recall context from previous messages within this conversation. Type 'exit' to end the session.

In [ ]:
while True:

    user_query = input("user: ")
    
    if user_query.lower().strip() == "exit":
        break
    print(f"User: {user_query}")
    response = agent_executor.invoke({"input":user_query})
    print(f"Assistant: {response['output']}")

## Summary

This notebook successfully demonstrated how to create a conversational agent with short-term memory.  
By integrating **Flotorch's session management** with **LangChain**, we achieved a **stateful interaction** where the agent can recall information from earlier in the conversation.

### Key Achievements

- **Session Persistence** 
  Configured `FlotorchLangChainSession` as the memory component for the `AgentExecutor`, enabling the agent to maintain context across turns.

- **Context-Aware Responses** 
  The agent successfully referenced previously mentioned details (e.g., the user's name) to provide more natural and relevant replies.

- **Streamlined Integration** 
  Demonstrated a clear and effective method to connect Flotorch's robust session infrastructure with LangChain’s powerful agent framework.
